In [1]:
import json
import os
from tqdm import tqdm

## load data

In [2]:
dataset_mode = 'original_' # 'original_', ''

### Line-level

In [4]:
datafile = os.path.join(f'../dataset/py150/line_completion/', f"{dataset_mode}test.json")
with open(datafile) as f:
    datas = f.readlines()
length = len(datas)
inputs = []
inputs_id = []
ids = []
# gts = []
for data in datas:
    data = json.loads(data.strip())
    ids.append(data["id"])
    inputs.append(data['input'])

### Token-level

In [5]:
# with open('../../CodeCompletion-token/dataset/javaCorpus/token_completion/test.txt') as f:
with open(f'../../CodeCompletion-token/dataset/py150/token_completion/{dataset_mode}test.txt') as f:
    datas = f.readlines()
with open(f'../../CodeCompletion-token/dataset/py150/token_completion/{dataset_mode}type_test.txt') as f:
    type_datas = f.readlines()

In [6]:
# with open('../../CodeCompletion-token/dataset/javaCorpus/token_completion/dev.txt') as f:
with open(f'../../CodeCompletion-token/dataset/py150/token_completion/{dataset_mode}dev.txt') as f:
    datas_dev = f.readlines()
with open(f'../../CodeCompletion-token/dataset/py150/token_completion/{dataset_mode}type_dev.txt') as f:
    type_datas_dev = f.readlines()

## Functions

In [8]:
def arrange_line_level_dict(ids, inputs, gts):
    line_dataset = []
    for i in range(len(inputs)):
        line = {}
        line["id"] = ids[i]
        line['input'] = inputs[i]
        line['gt'] = gts[i].strip()
        line_dataset.append(line)
    return line_dataset

In [9]:
def save_line_dataset(line_dataset, filename='gts.json'):
    with open(filename,"a") as f:
        for i in range(len(line_dataset)):
            json.dump(line_dataset[i], f)
            f.write("\n")

In [10]:
def create_repeat_types(tokens_list, types_lists, tokenizer, has_start_token=True):
    new_gts_type = []
    for j in tqdm(range(len(tokens_list))):
        gt_types = types_lists[j].split(' ')
        gt_tokens = tokenizer.tokenize(tokens_list[j].strip())
        new_gt_types = []
        i = 0
        next_type = False
        is_new_data = (i == 0 and not has_start_token)
        for tok in gt_tokens:
            if not is_new_data:
                if tok[0] == 'Ġ' or tok.startswith('<NUM_LIT') or next_type:
                    i+=1
                    next_type = False
                    if tok in ['<EOL>','<INDENT>','<DEDENT>'] or tok.startswith('<NUM_LIT'):
                        next_type = True
                elif tok in ['<EOL>','<INDENT>','<DEDENT>'] or tok.startswith('<NUM_LIT'):
                    next_type = True
                    i+=1
            is_new_data = False
            new_gt_types.append(gt_types[i])
        if new_gt_types[-2].strip() == '</s>':
            new_gt_types = new_gt_types[:-1]
        # if gt_types[i].strip() != '</s>' or tok.strip() != '</s>':
            # print(gt_types[i], tok)
            # print(j, 'Error')
            # break
        new_gts_type.append(' '.join(new_gt_types))
    return new_gts_type

## create line-level gts

In [11]:
data_inputs = []
data_types = []
data_gts = []
data_type_gts = []
for g,k in enumerate(ids):
    new_data = datas[k].split()
    gt_data = inputs[g].split()
    type_gt_data = type_datas[k].split()
    inp = []
    typ = []
    gt = []
    typ_gt = []
    j = 0
    for i in range(len(gt_data)):
        while new_data[j] in ['<INDENT>','<DEDENT>']:
            inp.append(new_data[j])
            typ.append(type_gt_data[j])
            j+=1
        if new_data[j] != gt_data[i]:
            print(f'error -> data: {new_data[j]} \t gt: {gt_data[i]}')
            break
        inp.append(new_data[j])
        typ.append(type_gt_data[j])
        j+=1
    for i in range(j, len(new_data)):
        if new_data[i] == '<EOL>':
            break
        gt.append(new_data[i])
        typ_gt.append(type_gt_data[i])
    data_inputs.append(" ".join(inp))
    data_types.append(" ".join(typ))
    data_gts.append(" ".join(gt))
    data_type_gts.append(" ".join(typ_gt))

In [14]:
data_gts_list = [d.split() for d in data_gts]
data_type_gts_list = [d.split() for d in data_type_gts]
data_inputs_list = [d.split() for d in data_inputs]
data_types_list = [d.split() for d in data_types]

In [10]:
line_dataset = arrange_line_level_dict(ids, data_inputs, data_gts)
type_line_dataset = arrange_line_level_dict(ids, data_types, data_type_gts)

In [12]:
# save_line_dataset(line_dataset, 'test.json')
# save_line_dataset(type_line_dataset, 'type_test.json')
# save_line_dataset(concat_type_code_line_dataset, 'concat_type_code_test.json')
# save_line_dataset(concat_code_type_line_dataset, 'concat_code_type_test.json')

### repeat type

In [1]:
from transformers import GPT2Tokenizer
import json
def get_special_tokens(path):
    lits = json.load(open(path))
    tokens = ["<STR_LIT>", "<NUM_LIT>", "<CHAR_LIT>"]
    for lit in lits["str"]:
        tokens.append(f"<STR_LIT:{lit}>")
    for lit in lits["num"]:
        tokens.append(f"<NUM_LIT:{lit}>")
    for lit in lits["char"]:
        tokens.append(f"<CHAR_LIT:{lit}>")
    return tokens
special_tokens = get_special_tokens('../dataset/py150/literals.json')
special_tokens.extend(['<NAME>', '<KEYWORD>', '<NUMBER>', '<STRING>', '<NEWLINE>', '<INDENT>', '<DEDENT>', '<LPAR>', '<RPAR>', '<LSQB>', '<RSQB>', '<COLON>', '<COMMA>', '<SEMI>', '<PLUS>', '<MINUS>', '<STAR>', '<SLASH>', '<VBAR>', '<AMPER>', '<LESS>', '<GREATER>', '<EQUAL>', '<DOT>', '<PERCENT>', '<LBRACE>', '<RBRACE>', '<EQEQUAL>', '<NOTEQUAL>', '<LESSEQUAL>', '<GREATEREQUAL>', '<TILDE>', '<CIRCUMFLEX>', '<LEFTSHIFT>', '<RIGHTSHIFT>', '<DOUBLESTAR>', '<PLUSEQUAL>', '<MINEQUAL>', '<STAREQUAL>', '<SLASHEQUAL>', '<PERCENTEQUAL>', '<AMPEREQUAL>', '<VBAREQUAL>', '<CIRCUMFLEXEQUAL>', '<LEFTSHIFTEQUAL>', '<RIGHTSHIFTEQUAL>', '<DOUBLESTAREQUAL>', '<DOUBLESLASH>', '<DOUBLESLASHEQUAL>', '<AT>', '<ATEQUAL>', '<RARROW>', '<ELLIPSIS>', '<ERRORTOKEN>'])
tokenizer = GPT2Tokenizer.from_pretrained('microsoft/CodeGPT-small-py', do_lower_case=False, sep_token='<EOL>', bos_token='<s>', eos_token='</s>', pad_token='<pad>', unk_token='<|UNKNOWN|>', additional_special_tokens=special_tokens)
# -adaptedGPT2

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [18]:
repeat_gts_type_test = create_repeat_types(data_gts, data_type_gts, tokenizer, has_start_token=False)
repeat_type_test = create_repeat_types(data_inputs, data_types, tokenizer)

100%|██████████| 10000/10000 [09:59<00:00, 16.68it/s]


In [84]:
# repeat_adapt_gts_type_test = create_repeat_types(data_gts, data_type_gts, tokenizer, has_start_token=False)
# repeat_adapt_type_test = create_repeat_types(data_inputs, data_types, tokenizer)

100%|██████████| 10000/10000 [10:00<00:00, 16.65it/s]


In [19]:
repeat_type_line_dataset = arrange_line_level_dict(ids, repeat_type_test, repeat_gts_type_test)
# repeat_adapt_type_line_dataset = arrange_line_level_dict(ids, repeat_adapt_type_test, repeat_adapt_gts_type_test)

In [20]:
save_line_dataset(repeat_type_line_dataset, 'original_type_repeat_test.json')
# save_line_dataset(repeat_adapt_type_line_dataset, 'type_adapt_repeat_test.json')

## Create Line-level for Dev set

### Analysis statistic of test dataset

In [98]:
def analysis_tokens(data_list):
    avg = 0
    min_len = len(data_list[0])
    max_len = len(data_list[0])
    min_idx = 0
    max_idx = 0
    for i in range(len(data_list)):
        avg += len(data_list[i])
        if len(data_list[i]) < min_len:
            min_len = len(data_list[i])  
            min_idx = i
        if len(data_list[i]) > max_len:
            max_len = len(data_list[i])  
            max_idx = i
    avg /= len(data_list)
    print('### Line-level Test datset ###')
    print('# input data')
    print(f'avg tokens: {avg}')
    print(f'min lens: {min_len}')
    print(f'max lens: {max_len}')
    return min_idx, max_idx

In [113]:
min_idx_input_test, max_idx_input_test = analysis_tokens(data_inputs_list)
print()
min_idx_gts_test, max_idx_gts_test = analysis_tokens(data_gts_list)

### Line-level Test datset ###
# input data
avg tokens: 502.2463
min lens: 11
max lens: 21333

### Line-level Test datset ###
# input data
avg tokens: 6.7219
min lens: 2
max lens: 46


In [114]:
datas_dev_list = [d.split() for d in datas_dev]
datas_test_list = [d.split() for d in datas]

In [116]:
min_idx_dev, max_idx_dev = analysis_tokens(datas_dev_list)
print()
min_idx_test, max_idx_test = analysis_tokens(datas_test_list)

### Line-level Test datset ###
# input data
avg tokens: 1066.773
min lens: 3
max lens: 38819

### Line-level Test datset ###
# input data
avg tokens: 897.6456
min lens: 3
max lens: 39799


### Process data

In [242]:
import random
random_seed = 42
ids_shuffle = [i for i in range(5000)]
random.Random(random_seed).shuffle(ids_shuffle)
data_inputs_dev = []
data_gts_dev = []
type_inputs_dev = []
type_gts_dev = []
ids_dev = []
for i in ids_shuffle:
    data_dev_list = datas_dev[i].split()
    type_dev_list = type_datas_dev[i].split()
    if len(data_dev_list) <= 15:
        continue
    tokens_len = random.randint(10, len(data_dev_list)-3)
    if data_dev_list[tokens_len] == '<EOL>':
        tokens_len += 1
    for j in range(tokens_len, len(data_dev_list)):
        if data_dev_list[j] == "<EOL>":
            break
    if j-tokens_len <= 1 or j-tokens_len > 100:
        continue
    data_inputs_dev.append(" ".join(data_dev_list[:tokens_len]))
    type_inputs_dev.append(" ".join(type_dev_list[:tokens_len]))
    data_gts_dev.append(" ".join(data_dev_list[tokens_len:j]))
    type_gts_dev.append(" ".join(type_dev_list[tokens_len:j]))
    ids_dev.append(i)
    if len(data_inputs_dev) == 2500:
        break

In [248]:
dev_line_dataset = arrange_line_level_dict(ids_dev, data_inputs_dev, data_gts_dev)
dev_type_line_dataset = arrange_line_level_dict(ids_dev, type_inputs_dev, type_gts_dev)

In [249]:
# save_line_dataset(dev_line_dataset, 'dev.json')
# save_line_dataset(dev_type_line_dataset, 'type_dev.json')

In [251]:
repeat_gts_type_dev = create_repeat_types(data_gts_dev, type_gts_dev, tokenizer, has_start_token=False)
repeat_type_dev = create_repeat_types(data_inputs_dev, type_inputs_dev, tokenizer)

100%|██████████| 2500/2500 [02:47<00:00, 14.93it/s]


In [253]:
repeat_adapt_gts_type_dev = create_repeat_types(data_gts_dev, type_gts_dev, tokenizer, has_start_token=False)
repeat_adapt_type_dev = create_repeat_types(data_inputs_dev, type_inputs_dev, tokenizer)

100%|██████████| 2500/2500 [02:46<00:00, 15.05it/s]


In [254]:
dev_repeat_type_line_dataset = arrange_line_level_dict(ids_dev, repeat_type_dev, repeat_gts_type_dev)
dev_repeat_adapt_type_line_dataset = arrange_line_level_dict(ids_dev, repeat_adapt_type_dev, repeat_adapt_gts_type_dev)

In [255]:
# save_line_dataset(dev_repeat_type_line_dataset, 'type_repeat_dev.json')
# save_line_dataset(dev_repeat_adapt_type_line_dataset, 'type_adapt_repeat_dev.json')

### Analysis len of dev dataset

In [244]:
data_gts_dev_list = [d.split() for d in data_gts_dev]
data_inputs_dev_list = [d.split() for d in data_inputs_dev]

In [245]:
min_idx_input_dev, max_idx_input_dev = analysis_tokens(data_inputs_dev_list)
print()
min_idx_gts_dev, max_idx_gts_dev = analysis_tokens(data_gts_dev_list)

### Line-level Test datset ###
# input data
avg tokens: 557.892
min lens: 10
max lens: 29961

### Line-level Test datset ###
# input data
avg tokens: 6.206
min lens: 2
max lens: 51


In [246]:
data_inputs_dev[min_idx_gts_dev]

"<s> import json <EOL> import base64 <EOL> import pytest <EOL> import jwt . exceptions <EOL> from django . test import TestCase <EOL> from rest_framework_jwt import utils <EOL> from rest_framework_jwt . compat import get_user_model <EOL> from rest_framework_jwt . settings import api_settings , DEFAULTS <EOL> User = get_user_model ( ) <EOL> def base64url_decode ( input ) : <EOL> <INDENT> rem = len ( input ) % <NUM_LIT:4> <EOL> if rem > <NUM_LIT:0> : <EOL> <INDENT> input += b'<STR_LIT:=>' * ( <NUM_LIT:4> - rem ) <EOL> <DEDENT> return base64 . urlsafe_b64decode ( input ) <EOL> <DEDENT> class UtilsTests ( TestCase ) : <EOL> <INDENT> def setUp ( self"

In [247]:
data_gts_dev[min_idx_gts_dev]

') :'